In [ ]:
import re
import string
import pandas as pd

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

data=pd.read_json('./data/news.json' ,lines=True) #download this file from: https://nuage.lip6.fr/s/XKkcWLAiDiykZ4D
df=data[["headline","short_description","date"]]
df=df.dropna()
df["content"]=df.apply(lambda row: row["headline"]+" "+row["short_description"] ,axis=1)
df=df[["content","date"]]
df['abstract'] = df['content'].apply(lambda x: remove_punct(x))
df['year'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['year'] = df['year'].dt.to_period('Y')
df=df.sort_values("year")
df=df.reset_index(drop=True)
df=df.dropna()
df=df[["abstract","year"]]
df=df.reset_index()

In [ ]:
from models.antm import ANTM
window_size=3
overlap=1

#take a random sample for example
dt=df.sample(n = 1000)
dt=dt.sort_values("year")
dt=dt.reset_index(drop=True)
dt=dt.reset_index()

evolving_topics=ANTM(dt,overlap,window_size,mode="data2vec",num_words=10)